### This notebook will only work with th data which already clean by the EDA notebook.

In [172]:
#base import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Model Import
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn import svm
#preprocessing import
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
#accuracy calculate import
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [175]:
ebay_clean= pd.read_csv('../data/cleaned/Ebay_cleaned.csv', index_col=0)
pd.set_option('display.max_columns', None)
ebay_clean.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,package_size,record_number,distance,handling_date,shipping_date,total_time,pay_year,pay_month,pay_date
0,1,25454,3,2019-03-27,0,0,3,5,97219,49040,13,28,1,2019-03-24,2019-03-29,5,1,1,3002,3,2,5,2019,3,24
1,0,6727381,2,2018-06-03,0,3,3,5,11415-3528,62521,0,20,1,2018-06-02,2018-06-05,0,4,2,1283,1,2,3,2018,6,2
2,1,18507,1,2019-01-08,0,4,3,5,27292,53010,1,20,1,2019-01-06,2019-01-10,9,4,3,1104,2,2,4,2019,1,6
3,1,4677,1,2018-12-18,0,0,3,5,90703,80022,1,36,1,2018-12-17,2018-12-21,8,4,4,1353,1,3,4,2018,12,17
4,1,4677,1,2018-07-28,0,0,3,5,90703,55070,1,25,1,2018-07-27,2018-07-30,3,4,5,2456,1,2,3,2018,7,27


In [176]:
ebay_clean.isna().sum()

b2c_c2c                      0
seller_id                    0
declared_handling_days       0
acceptance_scan_timestamp    0
shipment_method_id           0
shipping_fee                 0
carrier_min_estimate         0
carrier_max_estimate         0
item_zip                     0
buyer_zip                    0
category_id                  0
item_price                   0
quantity                     0
payment_datetime             0
delivery_date                0
weight                       0
package_size                 0
record_number                0
distance                     0
handling_date                0
shipping_date                0
total_time                   0
pay_year                     0
pay_month                    0
pay_date                     0
dtype: int64

In [111]:
ebay_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296420 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   b2c_c2c                    296420 non-null  int64 
 1   seller_id                  296420 non-null  int64 
 2   declared_handling_days     296420 non-null  int64 
 3   acceptance_scan_timestamp  296420 non-null  object
 4   shipment_method_id         296420 non-null  int64 
 5   shipping_fee               296420 non-null  int64 
 6   carrier_min_estimate       296420 non-null  int64 
 7   carrier_max_estimate       296420 non-null  int64 
 8   item_zip                   296420 non-null  object
 9   buyer_zip                  296420 non-null  object
 10  category_id                296420 non-null  int64 
 11  item_price                 296420 non-null  int64 
 12  quantity                   296420 non-null  int64 
 13  payment_datetime           296420 non-null  

In [112]:
X= ebay_clean.drop(columns=['acceptance_scan_timestamp',  'item_zip', 'buyer_zip','payment_datetime',  'delivery_date', 'record_number', 'total_time', 'shipping_date', 'handling_date'])
y= ebay_clean['total_time']

In [113]:
print(y.shape)
print(X.shape)

(296420,)
(296420, 16)


In [114]:
#split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Scale data


In [115]:
scaler= StandardScaler()
X_train_ss= scaler.fit_transform(X_train, y_train)
X_test_ss= scaler.transform(X_test)

In [116]:
y_true= y_test.values

In [117]:
def define_late(pred, y_actual= y_true):
    ontime= 0
    late= 0
    early= 0
    for i in range(len(pred)):
        if pred[i] == y_true[i]:
            ontime+=1
        elif pred[i]> y_true[i]:
            early+=1
        else:
            late+=1
        
    print(f'Ontime= {ontime}; {ontime/len(y_true)*100}')
    print(f'Late= {late}; {late/len(y_true)*100}')
    print(f'Early= {early}; {early/len(y_true)*100}')

Note:
pred > true-> early: 
pred = true-> ontime: 
pred < true-> late: 

#### LinearRegression model

In [171]:
#Initialize
linear_model= LinearRegression()
#fit model
linear_model.fit(X_train_ss, y_train)
linear_preds= linear_model.predict(X_test_ss)
linear_train_pred= linear_model.predict(X_train_ss)
linear_train_pred= np.round(linear_train_pred)
linear_preds= np.round(linear_preds)
linear_score_test=  linear_model.score(X_test_ss, y_test)
linear_score_train=  linear_model.score(X_train_ss, y_train)
linear_accuracy_test= accuracy_score(y_test, linear_preds)
linear_accuracy_train= accuracy_score(y_train, linear_train_pred)

In [119]:
print(f'Accuracy score test:  {linear_accuracy_test}')
print(f'Accuracy score train: {linear_accuracy_train}')

Accuracy score test:  0.22284258821941838
Accuracy score train: 0.2223576344376223


In [120]:
define_late(linear_preds)

Ontime= 13211; 22.284258821941837
Late= 17608; 29.701099790837326
Early= 28465; 48.014641387220834


3 window:

1-3 days: 1
4-7 days: 2
>7 days: 3

#### Ridge model

In [121]:
# Initialize
ridge_model= Ridge(solver='lsqr')

# fit
ridge_model.fit(X_train_ss, y_train)
ridge_preds= ridge_model.predict(X_test_ss)
ridge_preds= np.round(ridge_preds)
ridge_train_pred= ridge_model.predict(X_train_ss)
ridge_train_pred= np.round(ridge_train_pred)
ridge_score_test= ridge_model.score(X_test_ss, y_test)
ridge_score_train= ridge_model.score(X_train_ss, y_train)
ridge_accuracy_test= accuracy_score(y_test, ridge_preds)
ridge_accuracy_train= accuracy_score(y_train, ridge_train_pred)


In [122]:
print(f'Ridge Accuracy score test:  {ridge_accuracy_test}')
print(f'Ridge Accuracy score train: {ridge_accuracy_train}')

Ridge Accuracy score test:  0.222808852304163
Ridge Accuracy score train: 0.22234920045880846


In [123]:
define_late(ridge_preds)

Ontime= 13209; 22.2808852304163
Late= 17609; 29.702786586600094
Early= 28466; 48.0163281829836


In [124]:
Ontime= 13202; 22.26907766007692
Late= 17617; 29.716280952702245
Early= 28465; 48.014641387220834

48.014641387220834

#### XGboost

In [125]:
# initialize 
xg_boost= XGBRegressor()
#fit
xg_boost.fit(X_train_ss, y_train)
xg_pred= xg_boost.predict(X_test_ss)
xg_pred= np.round(xg_pred)
xg_train_pred= xg_boost.predict(X_train_ss)
xg_train_pred= np.round(xg_train_pred)
xg_train_accuracy=accuracy_score(y_train, xg_train_pred)
xg_test_accuracy= accuracy_score(y_test, xg_pred)
print(f'Train Score: {xg_boost.score(X_train_ss, y_train)}')
print(f'test score: {xg_boost.score(X_test_ss, y_test)}')
print(f'Test Accuracy : {xg_test_accuracy}')
print(f'Train Accuracy : {xg_train_accuracy}')

Train Score: 0.32032350030240453
test score: 0.1466805137403372
Test Accuracy : 0.24215639970312394
Train Accuracy : 0.246622191485055


In [126]:
define_late(xg_pred)

Ontime= 14356; 24.215639970312395
Late= 17107; 28.856015113690038
Early= 27821; 46.92834491599757


In [127]:
def evaluate_loss(preds, actual= y_true):
    early_loss, late_loss = 0,0 
    for i in range(len(preds)):
        if preds[i] < actual[i]:
            #early shipment
            early_loss += actual[i] - preds[i]
        elif preds[i] > actual[i]:
            #late shipment
            late_loss += preds[i] - actual[i]
    loss = (1/len(preds)) * (0.4 * (early_loss) + 0.6 * (late_loss))
    return loss

In [128]:
print(f'rigde lost= {evaluate_loss(ridge_preds)} -- linear loss= {evaluate_loss(linear_preds)} --xg lost= {evaluate_loss(xg_pred)}')

rigde lost= 0.7798326698603334 -- linear loss= 0.7798259226772822 --xg lost= 0.735129208555428


In [21]:
accuracy = [linear_accuracy_test, linear_accuracy_train, ridge_accuracy_test, ridge_accuracy_train, xg_test_accuracy, xg_train_accuracy]
columns=['linear test', 'linear train', 'ridge test', 'ridge train', 'xg test', 'xg train']

#### Neural Network

In [138]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GRU, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [23]:
#build model
tf.random.set_seed(123)
# Create a new sequential model
nn_model= keras.Sequential()
regularizer= keras.regularizers.l2(0.02)
#hidden layers
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
# nn_model.add(Dropout(0.2))
#output layer
nn_model.add(Dense(1))

#compile nn_model
nn_model.compile(
     optimizer=keras.optimizers.Adam(),
     loss=keras.losses.MeanSquaredError(),
     metrics=[keras.metrics.BinaryAccuracy()]
)

In [24]:
history= nn_model.fit(X_train_ss, y_train, epochs=70, verbose=0)

2023-06-18 15:47:01.588222: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [25]:
#evaluate
NN_train_accuracy=history.history['binary_accuracy'][-1] 
result = nn_model.evaluate(X_test_ss,y_test, verbose=0)
NN_pred = np.round(nn_model.predict(X_test_ss))

1853/1853 [==============================] - 0s 191us/step


In [112]:
define_late(NN_pred)

Ontime= 13171; 22.21678699143108
Late= 16814; 28.361783955198703
Early= 29299; 49.42142905337022


##### RNN


In [113]:
print(X_train_ss.shape, y_train.shape)

(237136, 17) (237136,)


In [43]:
#Make embedding
number_class= X_train_ss.shape[1]
embedding_dim= 8


In [46]:
tf.random.set_seed(123)

#define rnn
rnn_mode= keras.Sequential()
# add layers
embedding_layer=Embedding(number_class, embedding_dim)

rnn_mode.add(LSTM(64, activation='relu', input_shape=(X_train_ss.shape[1], 1)))

rnn_mode.add(Dense(64, activation= 'relu'))

#output layer
rnn_mode.add(Dense(1))

# Compile mode
rnn_mode.compile(
    loss='mean_squared_error', 
    optimizer=Adam(learning_rate=0.02),
    metrics='accuracy'
)

In [47]:
# fit model
rnn_history= rnn_mode.fit(X_train_ss, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
5929/5929 [==============================] - 19s 3ms/step - loss: 710.0778 - accuracy: 0.0306 - val_loss: 8.5887 - val_accuracy: 0.0305
Epoch 2/50
5929/5929 [==============================] - 17s 3ms/step - loss: 9.0117 - accuracy: 0.0307 - val_loss: 8.7712 - val_accuracy: 0.0305
Epoch 3/50
5929/5929 [==============================] - 17s 3ms/step - loss: 8.8567 - accuracy: 0.0307 - val_loss: 7.9580 - val_accuracy: 0.0305
Epoch 4/50
5929/5929 [==============================] - 17s 3ms/step - loss: 9.1137 - accuracy: 0.0307 - val_loss: 8.7007 - val_accuracy: 0.0305
Epoch 5/50
5929/5929 [==============================] - 18s 3ms/step - loss: 9.4174 - accuracy: 0.0307 - val_loss: 8.6986 - val_accuracy: 0.0305
Epoch 6/50
5929/5929 [==============================] - 17s 3ms/step - loss: 9.4165 - accuracy: 0.0307 - val_loss: 8.6974 - val_accuracy: 0.0305
Epoch 7/50
5929/5929 [==============================] - 18s 3ms/step - loss: 9.4171 - accuracy: 0.0307 - val_loss: 8.6978 - val_

In [48]:
rnn_train_accu= rnn_history.history['accuracy'][-1]
result= rnn_mode.evaluate(X_test_ss, y_test)
rnn_pred= np.round(rnn_mode.predict(X_test_ss))

1853/1853 [==============================] - 2s 833us/step


In [51]:
rnn_mode.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 64)                16896     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 21,121
Trainable params: 21,121
Non-trainable params: 0
_________________________________________________________________


In [50]:
print(f'accuracy_train= {rnn_train_accu}')
print(f'evaluate; ={result}')
print(f'predic= {rnn_pred}')
define_late(rnn_pred)

accuracy_train= 0.03068399801850319
evaluate; =[10.119983673095703, 0.030193643644452095]
predic= [[4.]
 [4.]
 [4.]
 ...
 [4.]
 [4.]
 [4.]]
Ontime= 11591; 19.55164968625599
Late= 16630; 28.0514135348492
Early= 31063; 52.39693677889481


### Pipeline - GridSearch


In [178]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
def build_regressor(optimizer = 'adam'):
  regressor = Sequential()
  regressor.add(Dense(units = 6 , activation= 'relu'))
  regressor.add(Dense(units = 6 , activation= 'relu'))
  regressor.add(Dense(units = 1 , activation= 'sigmoid'))
  regressor.compile(optimizer=optimizer , loss = 'mean_squared_error' , metrics=['accuracy'])
  return regressor
KC = KerasRegressor(build_fn=build_regressor)
parameters = {'batch_size' : [32, 64, 128],
          'epochs' : [10, 20, 30, 40, 50],
          'optimizer':['adam','rmsprop']}
grid_search = GridSearchCV(estimator=KC , 
param_grid=parameters,scoring='accuracy',cv=10)
grid_search.fit(X_train,y_train)

/var/folders/lv/tbn_js5s77ndnb22_v4_z6q80000gn/T/ipykernel_28814/1814552481.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  KC = KerasRegressor(build_fn=build_regressor)


Epoch 1/10
6670/6670 [==============================] - 2s 298us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 2/10
6670/6670 [==============================] - 2s 284us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 3/10
6670/6670 [==============================] - 2s 275us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 4/10
6670/6670 [==============================] - 2s 293us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 5/10
6670/6670 [==============================] - 2s 291us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 6/10
6670/6670 [==============================] - 2s 287us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 7/10
6670/6670 [==============================] - 2s 279us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 8/10
6670/6670 [==============================] - 2s 281us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 9/10
6670/6670 [==============================] - 2s 287us/step - loss: 18.2060 - accuracy: 0.0307
Epoch 10/10
742/742 [==============================] - 

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

6670/6670 [==============================] - 3s 289us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 2/10
6670/6670 [==============================] - 2s 285us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 3/10
6670/6670 [==============================] - 2s 287us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 4/10
6670/6670 [==============================] - 2s 290us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 5/10
6670/6670 [==============================] - 2s 285us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 6/10
6670/6670 [==============================] - 2s 285us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 7/10
6670/6670 [==============================] - 2s 287us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 8/10
6670/6670 [==============================] - 2s 283us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 9/10
6670/6670 [==============================] - 2s 292us/step - loss: 25.2771 - accuracy: 0.0019
Epoch 10/10
742/742 [==============================] - 0s 191us/st

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

...

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

1668/1668 [==============================] - 1s 298us/step - loss: 19.0020 - accuracy: 0.0281
Epoch 2/50
1668/1668 [==============================] - 0s 297us/step - loss: 18.3353 - accuracy: 0.0308
Epoch 3/50
1668/1668 [==============================] - 0s 296us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 4/50
1668/1668 [==============================] - 0s 294us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 5/50
1668/1668 [==============================] - 0s 294us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 6/50
1668/1668 [==============================] - 0s 293us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 7/50
1668/1668 [==============================] - 0s 295us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 8/50
1668/1668 [==============================] - 0s 296us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 9/50
1668/1668 [==============================] - 0s 291us/step - loss: 18.3352 - accuracy: 0.0308
Epoch 10/50
1668/1668 [==============================] - 0s 290us/

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan 0.02782783
 0.02778157        nan        nan        nan        nan        nan
        nan 0.02767614        nan        nan        nan        nan
 0.03065329        nan        nan        nan        nan 0.02782783
 0.02757072        nan        nan        nan 0.01884146        nan]
  warnings.warn(


3706/3706 [==============================] - 1s 297us/step - loss: 18.4358 - accuracy: 0.0303
Epoch 2/50
3706/3706 [==============================] - 1s 305us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 3/50
3706/3706 [==============================] - 1s 307us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 4/50
3706/3706 [==============================] - 1s 301us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 5/50
3706/3706 [==============================] - 1s 306us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 6/50
3706/3706 [==============================] - 1s 301us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 7/50
3706/3706 [==============================] - 1s 305us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 8/50
3706/3706 [==============================] - 1s 330us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 9/50
3706/3706 [==============================] - 1s 302us/step - loss: 18.3468 - accuracy: 0.0307
Epoch 10/50
3706/3706 [==============================] - 1s 307us/

GridSearchCV(cv=10,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x17a1d7bb0>,
             param_grid={'batch_size': [32, 64, 128],
                         'epochs': [10, 20, 30, 40, 50],
                         'optimizer': ['adam', 'rmsprop']},
             scoring='accuracy')

In [184]:
grid_search.best_params_

{'batch_size': 64, 'epochs': 50, 'optimizer': 'adam'}

In [146]:
import tensorflow as tf
tf.keras.backend.clear_session()
estimators= [
    ('normalise', StandardScaler()),
    ('model', LinearRegression())
] 
np.random.seed(123)
learning_rates = [0.1, 0.01, 0.001, 0.0001]

my_pipe= Pipeline(estimators)

grid= [
    # {
    #     'model': [Ridge()],
    #     'normalise':[StandardScaler()],
    #     'model__solver':['auto']
    # }
    {
        'model': [Sequential(),
            Sequential().add(Dense(units=64, activation='relu')),
            Sequential().add(Dense(units=32, activation='relu')),
            Sequential().add(Dense(units=1))
        ],
        'normalise': [StandardScaler()],
        'model__optimizer': [Adam(learning_rate=lr) for lr in learning_rates],
        'model__loss': ['mean_squared_error'],
        'model__epochs': [10, 20, 30],
        'model__batch_size': [32, 64, 128]
    }
    # {
    #     'model': [keras.Sequential()],
    #     'normalise': [StandardScaler()],
    #     'model__layer':[
    #             LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], 1)),
    #             Dense(units=64, activation='relu'),
    #             Dense(units=1)
    #         ],
    #     'model__optimizer': [Adam(learning_rate=lr) for lr in learning_rates],
    #     'model__epochs': [10, 20, 30, 40, 50],
    #     'model__loss': ['mean_squared_error'],
    #     'model__batch_size': [32, 64, 128]
    # },
    # {
    #     'model':[XGBRegressor()],
    #     'normalise':[StandardScaler()],
    #     'model__max_depth': range (2, 10, 1),
    #     'model__n_estimators': range(60, 220, 40),
    #     'model__learning_rate': [0.1, 0.01, 0.05]
    # }
]
gridCV= GridSearchCV(my_pipe, grid, cv=5)
fit_grid= gridCV.fit(X_train, y_train)

RuntimeError: Should only create a single instance of _DefaultDistributionStrategy

In [73]:
fit_grid.best_estimator_

Pipeline(steps=[('normalise', StandardScaler()), ('model', Ridge())])

In [74]:
fit_grid.best_params_

{'model': Ridge(), 'model__solver': 'auto', 'normalise': StandardScaler()}

In [75]:
fit_grid.best_score_

0.10871455118911615

In [76]:
fit_grid.predict(X_test)

array([3.64085683, 3.75509535, 4.01472428, ..., 3.71278943, 5.29679897,
       3.45039004])

In [77]:
define_late(np.round(fit_grid.predict(X_test)))

Ontime= 13202; 22.26907766007692
Late= 17617; 29.716280952702245
Early= 28465; 48.014641387220834
